In [5]:
conda install mat73

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - mat73

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import pandas as pd
import numpy as np
import mat73
import itertools



def data():
    Data=mat73.loadmat('GEpyramids_sigma0_kappa_1_to_5_by_25_normalized_reordered.mat', use_attrdict=True)
    return Data
def coord_matrix():
    
    
    Data = data()
    #gudhi.plot_persistence_diagram(data.X.h0_t20[2])
    
    f1=np.zeros([850,1])
    f2=np.zeros([850,1])
    f3=np.zeros([850,1])
    f4=np.zeros([850,1])
    f5=np.zeros([850,1])
    
    M= np.zeros([850,10])
    
    for k in range(2):
        for j in range(850):
            if k==0:
                H0_pers=Data.X.h0_t20_n[j]
            if k==1:
                H0_pers=Data.X.h1_t20_n[j]
            H0_pers=np.array(H0_pers[0])    
            p=[H0_pers[i,1]-H0_pers[i,0] for i in range(len(H0_pers))]
            
            p=np.array(p)
            
            b=H0_pers[:,0]
            d=H0_pers[:,1]
            b=np.array(b)
            d=np.array(d)
            d_max=np.max(d)
            #Carlson Coordinates
            f1[j]=np.sum(d*p)
            f2[j]=d_max*np.sum(p)-np.sum(d*p)
            f3[j]=np.sum(b**2*(d-b)**4)
            f4[j]=d_max**2*np.sum((1-d/d_max)**2*p**4)
            f5[j]=np.max(p)
        if k==0:
            M[:,0:5]=np.concatenate((f1,f2,f3,f4,f5),axis=1)
        else:
            M[:,5:10]=np.concatenate((f1,f2,f3,f4,f5),axis=1)

    return(M)




kappa=np.zeros([850,1])
for j in range(17):
    kappa[50*j:50*(j+1)]=np.array([[j*0.25+1]*50]).transpose()
    
C=coord_matrix()


a=[]
index=np.array(range(len(C[0])))

for L in range(len(index) + 1):
    for subset in itertools.combinations(index, L):
        a.append(subset)
        
a.pop(0)
AIC_values=[]
for i in range(len(a)):
    x = np.array(C[:,a[i]])
    x = sm.add_constant(x)

    model = sm.OLS(kappa, x).fit()

    AIC_values.append(np.array([a[i],model.aic]))


In [3]:
AIC_values=np.array(AIC_values)
minimum_index=np.argmin(AIC_values[:,1])
print(AIC_values[minimum_index])

[(0, 1, 2, 3, 5, 6, 7, 8, 9) 904.6811963066621]
